
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>


# Image Captioning Model
In the realm of multi-modal large language models (MLLMs), image-to-text and text-to-image models are among one of the most commonly seen applications. Can we generate a caption given an image and vice versa? We saw in the presentation that transformer architectures are general sequence-processing architectures that can process both images and text. In this lesson, we will learn how to build a vision-text transformer architecture yourself and compare the DIY-transformer performance against a pre-trained image captioning model. 

### ![Dolly](https://files.training.databricks.com/images/llm/dolly_small.png) Learning Objectives
1. Connect pretrained vision and text models as a single transformer architecture for image captioning
1. Train your image captioning model
1. Qualitatively inspect the quality of the caption generated
1. Compare the output with an pretrained image captioning model

## Classroom Setup

In [0]:
%run ../Includes/Classroom-Setup

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


Resetting the learning environment:
| enumerating serving endpoints...found 0...(0 seconds)
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/llm-foundation-models/v01-raw"

Validating the locally installed datasets:
| listing local files...(3 seconds)
| removing extra file: /models/models--gpt2/snapshots/11c5a3d5811f50298f278a704980280950aedb10/tokenizer.json...(0 seconds)
| fixed 1 issue...(3 seconds total)


Importing lab testing framework.



Using the "default" schema.

Predefined paths variables:
| DA.paths.working_dir: /dbfs/mnt/dbacademy-users/labuser4574468@vocareum.com/llm-foundation-models
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/labuser4574468@vocareum.com/llm-foundation-models/database.db
| DA.paths.datasets:    /dbfs/mnt/dbacademy-datasets/llm-foundation-models/v01-raw

Setup completed (9 seconds)

The models developed or used in this course are for demonstration and learning purposes only.
Models may occasionally output offensive, inaccurate, biased information, or harmful instructions.


We are going to use `sbu_captions` as our dataset to train our own image captioning model. 

`sbu_captions` contains 1 million pairs of image urls and captions. Visit [this link](https://huggingface.co/datasets/sbu_captions) to view their dataset on Hugging Face Datasets hub.

In [0]:
# Required Libraries:
# - `datasets`: HuggingFace's datasets library for loading datasets

# Import the 'load_dataset' function from the 'datasets' library
from datasets import load_dataset

# Load the SBU Captions dataset and shuffle it using a seed value of 42
# The 'split="train"' argument specifies that only the training split of the dataset is loaded
# The 'shuffle(seed=42)' function shuffles the dataset in a reproducible manner using the specified seed
data = load_dataset("sbu_captions", split="train").shuffle(seed=42)

# Display the shuffled dataset
# Useful for debugging and initial exploration of the dataset
data


/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:27: UserWarning: This dataset can not be stored in DBFS because either `cache_dir` or the environment variable `HF_DATASETS_CACHE` is set to a non-DBFS path. If this cluster restarts, all saved dataset information will be lost.
  warnings.warn(


/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Dataset sbu_captions downloaded and prepared to /root/.cache/huggingface/datasets/sbu_captions/default/0.0.0/0c994175c0b31f384f0b9a6aa51c5c9969c468df4c3d33a52074686ea51ee684. Subsequent calls will reuse this data.


Dataset({
    features: ['image_url', 'user_id', 'caption'],
    num_rows: 1000000
})

## Initialize an image-to-text model

In the following section, we are going to use [Vision Encoder Decoder Model](https://huggingface.co/docs/transformers/model_doc/vision-encoder-decoder) to initialize our image-to-text model. The encoder would be a transformer-based vision model to process the images and the decoder is a language model to generate the caption. Then, we will connect both the vision and language model together and further train the vision-language model on a subset of `sbu_captions`. 



### Define data processing function

First, we will need to have a function to properly process our training dataset that contains the image urls and respective captions.

In [0]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import requests
from io import BytesIO

# Required Libraries:
# - torch: For tensor operations
# - PIL: For image processing
# - requests: For fetching images from URLs
# - HuggingFace Transformers: For tokenization and feature extraction

class ProcessDataset(Dataset):
    """
    A class to process a dataset for a multimodal (vision + text) model.
    
    Parameters:
    - df (Pandas DataFrame): A DataFrame containing 'image_url' and 'caption' columns.
    - tokenizer: A tokenizer object for text processing.
    - feature_extractor: A feature extraction object for image processing.
    - decoder_max_length (int): Maximum length of output caption.
    """
    
    def __init__(self, df, tokenizer, feature_extractor, decoder_max_length=20):
        self.df = df
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.decoder_max_length = decoder_max_length

    def __len__(self):
        """Returns the length of the dataset."""
        return len(self.df)

    def __getitem__(self, idx):
        """
        Fetch and preprocess the image and text at a given index.
        
        Parameters:
        - idx (int): Index of the data to fetch.
        
        Returns:
        - encoding (dict): A dictionary containing processed image and text data.
        """
        
        # Fetch image URL and caption from DataFrame
        img_path = self.df["image_url"][idx]
        caption = self.df["caption"][idx]
        
        # Fetch image from URL
        response = requests.get(img_path)
        # TODO: Add error handling for failed requests
        
        # Process image
        image = Image.open(BytesIO(response.content))
        
        # Extract features from the image
        # 'pt' indicates that PyTorch tensors are returned
        pixel_values = self.feature_extractor(image, return_tensors="pt").pixel_values
        
        # Process caption text
        labels = self.tokenizer(
            caption,
            truncation=True,
            padding="max_length",
            max_length=self.decoder_max_length
        ).input_ids
        
        # Prepare output encoding
        encoding = {
            "pixel_values": pixel_values.squeeze(),  # Remove singleton dimensions
            "labels": torch.tensor(labels)
        }
        
        return encoding


### Initialize tokenizer and image feature extractor

Next, we will initialize our tokenizer to process text and feature extractor to process images respectively. After this, we are ready to pass our training dataset for processing.

In [0]:
# Required Libraries:
# - transformers: HuggingFace's Transformers library for NLP and computer vision models

# Import the necessary classes from the 'transformers' library
from transformers import GPT2TokenizerFast, ViTFeatureExtractor

# Initialize the GPT-2 tokenizer using a pre-trained model
# The 'cache_dir' argument specifies the directory to store the downloaded pre-trained model
tokenizer = GPT2TokenizerFast.from_pretrained(
    "gpt2", 
    cache_dir=DA.paths.datasets+"/models"
)

# Add a padding token to the GPT-2 tokenizer
# GPT-2 originally does not have a pad token, so we set it to be the same as the EOS (end-of-sentence) token
tokenizer.pad_token = tokenizer.eos_token

# Initialize the Vision Transformer (ViT) feature extractor using a pre-trained model
# The 'cache_dir' argument specifies the directory to store the downloaded pre-trained model (Vision Transformer (base-sized model))
feature_extractor = ViTFeatureExtractor.from_pretrained(
    "google/vit-base-patch16-224-in21k", 
    cache_dir=DA.paths.datasets+"/models"
)

/databricks/python/lib/python3.10/site-packages/huggingface_hub/file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in /dbfs/mnt/dbacademy-datasets/llm-foundation-models/v01-raw/models. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
  warnings.warn(message)
/databricks/python/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [0]:
# Required Libraries:
# Assume that the 'ProcessDataset' class, 'tokenizer', and 'feature_extractor' have been previously defined.
# 'data' is a dataset loaded via some data loading mechanism (e.g., HuggingFace datasets library).

# Initialize the training dataset using the ProcessDataset class
# - df=data[:2000]: We are using the first 2000 samples from the 'data' DataFrame for training.
# - tokenizer=tokenizer: We pass in the previously initialized GPT-2 tokenizer for text tokenization.
# - feature_extractor=feature_extractor: We pass in the previously initialized ViT feature extractor for image processing.
train_dataset = ProcessDataset(
    df=data[:2000],  # Use the first 2000 rows of the DataFrame 'data' for the training dataset
    tokenizer=tokenizer,  # Use the GPT-2 tokenizer defined earlier for text tokenization
    feature_extractor=feature_extractor  # Use the ViT feature extractor defined earlier for image feature extraction
)


### Using VisionEncoderDecoder 

Here, we will finally use `VisionEncoderDecoder` to connect our pretrained image and text models of choice. 

You might see in the output that some weights of the GPT2 model are not initialized from the model checkpoint; from HuggingFace: `You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.` For the best performance, we should ideally fine-tune this decoder on our own dataset separately and load the fine-tuned decoder. However, for simplicity's sake, we are simply going to use the model as is, and fine-tune the image-captioning model as a whole. 

In [0]:
# Required Libraries:
# - transformers: HuggingFace's Transformers library for creating encoder-decoder models.

# Import the VisionEncoderDecoderModel class from the 'transformers' library
from transformers import VisionEncoderDecoderModel

# Initialize the VisionEncoderDecoderModel using pre-trained models for both the encoder and decoder
# - encoder_pretrained_model_name_or_path: Specifies the pre-trained model to use for the encoder (Vision Transformer).
# - decoder_pretrained_model_name_or_path: Specifies the pre-trained model to use for the decoder (GPT-2).
# - tie_encoder_decoder: If set to True, ties the encoder and decoder weights together for more efficient training.
# - cache_dir: Specifies the directory to cache the downloaded pre-trained models.
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_pretrained_model_name_or_path="google/vit-base-patch16-224-in21k",  # Use Vision Transformer as the encoder
    decoder_pretrained_model_name_or_path="gpt2",  # Use GPT-2 as the decoder
    tie_encoder_decoder=True,  # Tie encoder and decoder parameters for more efficient training
    cache_dir=DA.paths.datasets+"/models"  # Cache the downloaded models in the specified directory
)


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.5.crossattention.q_attn.weight', 'h.6.ln_cross_attn.weight', 'h.5.crossattention.c_proj.weight', 'h.10.crossattention.c_proj.bias', 'h.5.crossattention.masked_bias', 'h.6.crossattention.c_attn.weight', 'h.3.crossattention.masked_bias', 'h.11.crossattention.c_proj.bias', 'h.11.crossattention.bias', 'h.5.ln_cross_attn.weight', 'h.0.ln_cross_attn.weight', 'h.11.crossattention.masked_bias', 'h.1.crossattention.q_attn.weight', 'h.0.crossattention.masked_bias', 'h.3.crossattention.bias', 'h.9.crossattention.masked_bias', 'h.5.crossattention.bias', 'h.10.crossattention.c_attn.weight', 'h.6.crossattention.c_proj.weight', 'h.7.ln_cross_attn.weight', 'h.9.crossattention.c_attn.weight', 'h.1.crossattention.c_attn.weight', 'h.9.ln_cross_attn.weight', 'h.0.crossattention.c_proj.bias', 'h.8.crossattention.masked_bias', 'h.7.crossattention.c_proj.bias', 'h.7.crossattention.masked_bias

In [0]:
# Update GPT-2 Configuration for Encoder-Decoder Model:
# GPT-2 uses bos (beginning of sentence) and eos (end of sentence) tokens,
# but it doesn't natively support decoder_start or pad tokens. We configure these manually.

# Set the decoder's start token ID to be the same as the bos_token_id from the tokenizer
# This is used to initiate the decoding process.
model.config.decoder_start_token_id = tokenizer.bos_token_id
# Set the padding token ID to be the same as the pad_token_id from the tokenizer
# Padding tokens are used to fill in sequences to a uniform length.
model.config.pad_token_id = tokenizer.pad_token_id
# Explicitly set the end-of-sentence token ID, even though it's usually set by default in GPT-2
# This is used to signify the end of a generated sequence.
model.config.eos_token_id = tokenizer.eos_token_id

# Additional Configuration Adjustments:

# Synchronize the vocab size between the model configuration and the decoder
# Ensures that the decoder uses the same vocabulary size as defined in its own configuration.
model.config.vocab_size = model.config.decoder.vocab_size
# Enable early stopping in the beam search to improve decoding efficiency
# Stops the search when at least `num_beams` sentences are finished per batch.
model.config.early_stopping = True
# Set no_repeat_ngram_size to prevent the repetition of n-grams within the generated text
# An n-gram of size 3 means any sequence of 3 words cannot be repeated in the generated text.
model.config.no_repeat_ngram_size = 3
# Apply a length penalty for longer sequences in the beam search
# A factor of 2.0 discourages the model from generating overly short sequences.
model.config.length_penalty = 2.0

# Decoder-Specific Settings:

# Set the number of beams for beam search in the decoder
# More beams increases the chances of finding a more accurate sequence but is computationally expensive.
model.decoder.num_beams = 4
# Set the maximum sequence length for the decoder's output
# Limits the length of generated sequences to 20 tokens.
model.decoder.max_length = 20


## Train image-captioning model

Recall that you have seen `data_collator` in Module 2 before. 

As a refresher, data collators help us form batches of inputs to pass in to the model for training. Go [here](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#data-collator) for documentation.

In [0]:
from transformers import Trainer, TrainingArguments
from transformers import default_data_collator
import os

BATCH_SIZE = 16
TRAIN_EPOCHS = 20

output_directory = os.path.join(DA.paths.working_dir, "captioning_outputs")

training_args = TrainingArguments(
    output_dir=output_directory,
    per_device_train_batch_size=BATCH_SIZE,
    do_train=True,
    num_train_epochs=TRAIN_EPOCHS, # number of passes to see the entire dataset 
    overwrite_output_dir=True,
    no_cuda=True, # Not using GPU
    dataloader_pin_memory=False # this specifies whether you want to pin memory in data loaders or not
)

trainer = Trainer(
    tokenizer=feature_extractor,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=default_data_collator,
)

Note: You might notice that this cells triggers a whole new MLflow run. [MLflow](https://mlflow.org/docs/latest/index.html) is an open source tool that helps to manage end-to-end machine learning lifecycle, including experiment tracking, ML code packaging, and model deployment. You can read more about [LLM tracking here](https://mlflow.org/docs/latest/llm-tracking.html).

In [0]:
# Same errors begin here because of stupid fetching flickr photos
# ConnectTimeout: HTTPConnectionPool(host='static.flickr.com', port=80): Max retries exceeded with url: /33/205540567_b8cab77901.jpg (Caused by ConnectTimeoutError(, 'Connection to static.flickr.com timed out. (connect timeout=None)'))

trainer.train()

---------------------------------------------------------------------------
TimeoutError                              Traceback (most recent call last)
File /databricks/python/lib/python3.10/site-packages/urllib3/connection.py:174, in HTTPConnection._new_conn(self)
    173 try:
--> 174     conn = connection.create_connection(
    175         (self._dns_host, self.port), self.timeout, **extra_kw
    176     )
    178 except SocketTimeout:

File /databricks/python/lib/python3.10/site-packages/urllib3/util/connection.py:95, in create_connection(address, timeout, source_address, socket_options)
     94 if err is not None:
---> 95     raise err
     97 raise socket.error("getaddrinfo returns an empty list")

File /databricks/python/lib/python3.10/site-packages/urllib3/util/connection.py:85, in create_connection(address, timeout, source_address, socket_options)
     84     sock.bind(source_address)
---> 85 sock.connect(sa)
     86 return sock

TimeoutError: [Errno 110] Connection timed out



## Generate caption from an image

Now, let's try generating caption from a randomly picked image below.

In [0]:
# Import the 'requests' library for fetching image data and 'Image' for displaying the image
# Assume these libraries have been imported earlier in the code

# Select a test image from the dataset at index 2021
test_img = data[2021]
# Extract the image URL from the test image data
test_img_path = test_img["image_url"]
# Fetch the image content from the URL
test_img_response = requests.get(test_img_path)
# Open and read the image using PIL (Python Imaging Library)
test_image = Image.open(BytesIO(test_img_response.content))
# Display the test image
# The 'display' function is assumed to be imported from a library like IPython
display(test_image)


---------------------------------------------------------------------------
TimeoutError                              Traceback (most recent call last)
File /databricks/python/lib/python3.10/site-packages/urllib3/connection.py:174, in HTTPConnection._new_conn(self)
    173 try:
--> 174     conn = connection.create_connection(
    175         (self._dns_host, self.port), self.timeout, **extra_kw
    176     )
    178 except SocketTimeout:

File /databricks/python/lib/python3.10/site-packages/urllib3/util/connection.py:95, in create_connection(address, timeout, source_address, socket_options)
     94 if err is not None:
---> 95     raise err
     97 raise socket.error("getaddrinfo returns an empty list")

File /databricks/python/lib/python3.10/site-packages/urllib3/util/connection.py:85, in create_connection(address, timeout, source_address, socket_options)
     84     sock.bind(source_address)
---> 85 sock.connect(sa)
     86 return sock

TimeoutError: [Errno 110] Connection timed out



In [0]:
# Generate features from the test image using the feature extractor
# "pt" just mean Pytorch
image_features = feature_extractor(test_image, return_tensors="pt").pixel_values

# Use the model to generate a caption for the image
generated_token_ids = trainer.model.generate(image_features)[0]

# Decode the generated token IDs to a human-readable caption
caption = tokenizer.decode(generated_token_ids)

# Print a separator and the generated caption
print("--" * 20)
print(caption)


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1542565039235822>:3
      1 # Generate features from the test image using the feature extractor
      2 # "pt" just mean Pytorch
----> 3 image_features = feature_extractor(test_image, return_tensors="pt").pixel_values
      5 # Use the model to generate a caption for the image
      6 generated_token_ids = trainer.model.generate(image_features)[0]

NameError: name 'test_image' is not defined

Hmm, you can see that the caption is not really relevant to the image above. What went wrong? There are two possible reasons: 
- Parts of our text decoder weights were not loaded from the pretrained checkpoints. So the best approach would have been to train the decoder separately on the training dataset first and load the fine-tuned decoder. 
- Our image-captioning model needs more fine-tuning time! Try increasing the # of epochs, # of training data samples, and adjust other model hyperaparameters if you'd like. 

But now you learned how to plug in your own transformer-based models and connect them as an encoder-decoder model! It would definitely be much easier if we use an existing image captioning model instead. Let's do that next!  

## What if we use an existing image captioning model instead? 

Will the caption generated improve and be more relevant? Let's find out! 

We will be using a new model called `BLIP`, which stands for Bootstrapping Language-Image Pre-training. It's a modeling approach to unify vision-language understanding and generation by [Li et al 2022](https://arxiv.org/abs/2201.12086). It stands out from other vision-language model because: 

>  Most existing pre-trained models only excel in either understanding-based tasks or generation-based tasks. [...] BLIP effectively utilizes the noisy web data by bootstrapping the captions, where a captioner generates synthetic captions and a filter removes the noisy ones.

The excerpt above is from the paper's abstract. You can also read briefly about BLIP in the [HuggingFace documentation](https://huggingface.co/Salesforce/blip-image-captioning-base).

In [0]:
from transformers import BlipProcessor, BlipForConditionalGeneration

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base", cache_dir=DA.paths.datasets+"/models")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base", cache_dir=DA.paths.datasets+"/models")

# conditional image captioning
# in many of the initial vision-language models, adding a prefix text like below "a photo of " is crucial for models to do well
# the addition of the prefix text makes the caption generation "conditional"
text = "a photo of"
inputs = blip_processor(test_image, text, return_tensors="pt")

conditional_output = blip_model.generate(**inputs)
print("Conditional output: ", blip_processor.decode(conditional_output[0], skip_special_tokens=True))

# unconditional image captioning
# in newer model iterations, researchers have found improvements to remove the need of adding a prefix text 
# therefore, the caption generation is "unconditional"
# notice that the `text` field is no longer filled out (it's now optional)
inputs = blip_processor(test_image, return_tensors="pt")

unconditional_output = blip_model.generate(**inputs)
print("Unconditional output: ", blip_processor.decode(unconditional_output[0], skip_special_tokens=True))

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1542565039235825>:10
      6 # conditional image captioning
      7 # in many of the initial vision-language models, adding a prefix text like below "a photo of " is crucial for models to do well
      8 # the addition of the prefix text makes the caption generation "conditional"
      9 text = "a photo of"
---> 10 inputs = blip_processor(test_image, text, return_tensors="pt")
     12 conditional_output = blip_model.generate(**inputs)
     13 print("Conditional output: ", blip_processor.decode(conditional_output[0], skip_special_tokens=True))

NameError: name 'test_image' is not defined

Now that you have seen how to generate caption from images, we are going to get our hands dirty to play with videos in the lab! You will learn how to perform zero-shot video classification. 

&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>